In [1]:
import re

def embeddings_path(dataset_name, embedding_size, bpe=False):
    if bpe:
        return f"embeddings/{dataset_name}_embeddings_{embedding_size}_bpe.txt"
    return f"embeddings/{dataset_name}_embeddings_{embedding_size}.txt"

def tokenize_text(input_file, output_file, tokenizer, regex, new_line_marker='\n', max_sentences=None):
    sentences = []
    with open(input_file, 'r', encoding='utf-8') as fin, \
         open(output_file, 'w', encoding='utf-8') as fout:
        pattern = re.compile(regex, re.UNICODE)
        for line in fin:
            tokens = pattern.findall(line.lower())
            if not tokens:
                continue
            if tokenizer == 'word':
                sentences.append(tokens)
                sentences[-1].append(new_line_marker + ' ')
            elif tokenizer == 'char':
                words = []
                for token in tokens:
                    token_chars = list(token)
                    token_chars.append('</w>')
                    tokenized_word = ' '.join(token_chars)
                    words.append(tokenized_word)
                sentences.append(words)
                sentences[-1].append(new_line_marker + ' ')
            fout.write(" ".join(sentences[-1]))
            if max_sentences is not None and len(sentences) >= max_sentences:
                break
    return sentences

def load_sentences(file_path):
    sentences = []
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            tokens = line.strip().split()
            sentences.append(tokens)
    return sentences


In [2]:
dataset_path = "datasets/harrypotter.txt"
# dataset_path = "datasets/chan_dialogues.txt"
# dataset_path = "datasets/war.txt"
dataset_name = dataset_path.split('/')[-1].split('.')[0]

embeddings_d = [100, 500, 1000]
max_vocab_size = 120000

bpe = True

# pattern = r'\w+|[^\w\s]' # со знаками препинания
pattern =  r"\w+"        # только слова   
tokenizer = 'char' if bpe else 'word'
tokenize_text(dataset_path, f"sentences/{dataset_name}_sentences_{tokenizer}.txt", tokenizer=tokenizer, regex=pattern, new_line_marker='</n> ')
sentences = load_sentences(f"sentences/{dataset_name}_sentences_{tokenizer}.txt")

total_words = sum(len(sentence) for sentence in sentences)
print(f"Total number of words: {total_words}")
print(f"Total number of sentences: {len(sentences)}")

Total number of words: 6000812
Total number of sentences: 1


In [13]:
import subprocess

if bpe:
    subprocess.run(["./bpe_tokenizer",
                    f"sentences/{dataset_name}_sentences_{tokenizer}.txt",
                    f"sentences/{dataset_name}_sentences_bpe.txt",
                   '2800'
                   ], check=True)

In [3]:
if bpe:
    sentences = load_sentences(f"sentences/{dataset_name}_sentences_bpe.txt")
    total_words = sum(len(sentence) for sentence in sentences)
    print(f"Total number of words after BPE: {total_words}")

Total number of words after BPE: 1417078


In [ ]:
import subprocess

for d in embeddings_d:
    subprocess.run(["./generate_embeddings",            # executable
                    f"sentences/{dataset_name}_sentences_{'bpe' if bpe else 'word'}.txt",       # path to tokenized sentences
                    embeddings_path(dataset_name, d, bpe=bpe),   # path to save embeddings to
                    str(d),                             # word vec dim
                    str(max_vocab_size),                # max size of vocabulary
                    '5',                                # windows size
                    '5',                                # num negative examples
                    '5',                                # num epoch
                    '0.025'                             # learning rate
                    ], check=True)

Текст: sentences/harrypotter_sentences_word.txt
Эмбеддинги: embeddings/harrypotter_embeddings_100.txt
Размер эмбеддингов (d): 100
Макс. размер словаря: 120000
Размер окна: 5
Количество отрицательных примеров: 5
Количество эпох: 5
Скорость обучения (lr): 0.025

Загрузка текста и создание словаря...
Текст прочитан. (0.120547 сек)
Количество строк: 1
Количество уникальных слов: 22054

Словарь создан. (0.128097 сек)
Размер словаря: 22054
Макс. размер словаря: 120000
Примеры словаря: 
the (52227) -> 0; </n> (38195) -> 1; and (27810) -> 2; to (27129) -> 3; he (22260) -> 4; of (22005) -> 5; a (21094) -> 6; harry (18367) -> 7; was (15689) -> 8; you (14667) -> 9; 

Предложения закодированы. (0.179729 сек)
Количество закодированных предложений: 1
Примеры закодированных предложений: 
the </n> and to he of a harry was you 
0 1 2 3 4 5 6 7 8 9 

Эпоха 1 | Средняя ошибка: 1.15552 | Время: 25.9511 сек | Общее время: 26.2438 сек
Эпоха 2 | Средняя ошибка: 0.873556 | Время: 23.0976 сек | Общее время: 49

In [4]:
import numpy as np

def load_embeddings(file_path):
    with open(file_path, encoding='utf-8') as f:
        header = f.readline().strip().split()
        vocab_size = int(header[0])
        d = int(header[1])
        embeddings = np.zeros((vocab_size, d), dtype=np.float32)
        word2index = {}
        index2word = []
        for i, line in enumerate(f):
            parts = line.strip().split()
            if len(parts) != d + 1:
                continue 
            word = parts[0]
            vector = np.array(parts[1:], dtype=np.float32)
            embeddings[i] = vector
            word2index[word] = i
            index2word.append(word)
    return embeddings, word2index, index2word

# embeddings_file = embeddings_path(dataset_name, 100)  
# embeddings, word2index, index2word = load_embeddings(embeddings_file)
# print("Загруженная матрица эмбеддингов:", embeddings.shape)
# print("Примеры слов:", list(word2index.keys())[:10])

# print("Примеры векторов:", list(embeddings)[:3])


In [5]:
def generate_training_examples_from_sentence(sentence, L):
    """
    Генерирует обучающие примеры (контекст, целевой токен) из предложения.
    где контекст — это L последовательных токенов, а целевой токен — следующий за ними.
    """
    if len(sentence) < L + 1:
        return  # Недостаточно токенов для формирования хотя бы одного примера.
    for i in range(len(sentence) - L):
        context = sentence[i : i + L]
        target = sentence[i + L]
        yield context, target
        
def batch_generator_emb(tokenized_sentences, word2index, embeddings, L, batch_size):
    while True:
        X, y = [], []
        for sentence in tokenized_sentences:
            for context, target in generate_training_examples_from_sentence(sentence, L):
                X.append([word2index.get(w, 0) for w in context])
                y.append(word2index.get(target, 0))
                if len(X) >= batch_size:
                    X_emb = embeddings[X]
                    yield X_emb, np.array(y, dtype=np.int32)
                    X, y = [], []  # Очищаем батч
        if X:  # Возвращаем остаток батча
            X_emb = embeddings[X]
            yield X_emb, np.array(y, dtype=np.int32)
                


In [ ]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.backend import clear_session as clear_keras_session
from tensorflow.keras.callbacks import EarlyStopping

L = 5    # Длина контекста для предсказания
dh = 500  # Размер скрытого слоя
epochs = 10

batch_size = 512

steps_per_epoch = np.ceil(total_words / batch_size).astype(int)

for d in embeddings_d:
    embeddings_file = embeddings_path(dataset_name, d, bpe=bpe)
    embeddings, word2index, index2word = load_embeddings(embeddings_file)

    vocab_size = len(word2index)

    print(f"Размер словаря: {vocab_size}, размер эмбеддинга: {d}")
    print(f"Всего слов: {total_words}, шагов на эпоху: {steps_per_epoch}")

    clear_keras_session()  # Очистка сессии Keras

    model = Sequential([ 
        layers.Input(shape=(L,d)), # 
        layers.Flatten(),
        layers.Dense(dh, activation='relu'),
        layers.Dense(vocab_size, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    # Обучаем модель предсказания
    model.fit(batch_generator_emb(sentences, word2index, embeddings, L, batch_size), 
            epochs=epochs, 
            steps_per_epoch=steps_per_epoch,
            callbacks=[EarlyStopping(monitor='loss', patience=3)])
    
    # Сохраняем модель
    model.save(f"models/{dataset_name}_model_{d}_{'bpe' if bpe else 'word'}_emb.keras")

Размер словаря: 2813, размер эмбеддинга: 100
Всего слов: 1417078, шагов на эпоху: 2768


I0000 00:00:1742661839.473690  657034 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1216 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2813)           │     1,409,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,659,813 (6.33 MB)

...

I0000 00:00:1742661840.536259  657265 service.cc:152] XLA service 0x7e02dc004700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742661840.536297  657265 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-03-22 19:44:00.556298: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742661840.637674  657265 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-22 19:44:01.572318: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_102', 108 bytes spill stores, 108 bytes spill loads

2025-03-22 19:44:02.070296: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusi

  45/2768 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.0240 - loss: 7.2622      

I0000 00:00:1742661849.625873  657265 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1901/2768 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0714 - loss: 6.1717

KeyboardInterrupt: 

In [8]:
# self embeddings layer

import numpy as np
import random

from tensorflow.keras.models import load_model
from tensorflow.keras.backend import clear_session as clear_keras_session

def evaluate_perplexity(model, test_sentence, L, word2index, embeddings):
    """
    Для каждого окна длины L модель предсказывает вероятность истинного следующего слова
    perplexity = exp( -(1/N) * sum(log(P(w_i|context)))
    где N - количество предсказаний, а P(w_i|context) - вероятность истинного слова
    """
    total_log_prob = 0.0
    count = 0

    test_tokens = test_sentence.lower().split()
    test_indices = [word2index.get(word, 0) for word in test_tokens]
    
    test_indices_emb = embeddings[test_indices]
    
    # Если тестовая последовательность слишком короткая возвращаем бесконечность
    if len(test_indices_emb) <= L:
        return float('inf')

    # Проходим по тестовой последовательности, формируя окна длины L
    for i in range(len(test_indices_emb) - L):
        context = test_indices_emb[i:i+L]         # Контекстное окно из L токенов
        true_word = test_indices[i+L]           # Истинное следующее слово
        
        # Получаем распределение вероятностей для следующего слова
        pred = model.predict(np.array([context]), verbose=0)[0]
        # Берем вероятность истинного слова; добавляем маленькую константу для стабильности log
        prob = pred[true_word] + 1e-10
        
        # Суммируем логарифмы вероятностей
        total_log_prob += np.log(prob)
        count += 1

    if count == 0:
        return float('inf')
        
    # Вычисляем среднее логарифмическое значение
    avg_log_prob = total_log_prob / count
    # Перплексия – экспонента от отрицательной средней логарифмической вероятности
    perplexity = np.exp(-avg_log_prob)
    return perplexity


# Длина контекстного окна для предсказания следующего слова
L = 5  
# Длина тестового предложения
S = 15
# Формируем один длинный список из всех слов из файла (load_sentences должна возвращать список предложений, где каждое предложение - список токенов)
sentences_unpacked = []
for sentence in load_sentences(f"sentences/{dataset_name}_sentences_{'bpe' if bpe else 'word'}.txt"):
    sentences_unpacked.extend(sentence) 

# Выбираем случайное окно в общем списке
i = random.randint(0, len(sentences_unpacked) - S - 1)
# Берем окно из L+1 слов (последнее слово будет истинным следом за контекстом)
test_sentence = " ".join(sentences_unpacked[i:i+S+1])
# test_sentence = 'my</w> name</w> is</w>'
print(f"Тестовое предложение: '{test_sentence}'")

# Преобразуем предложение в список токенов (слова приводятся к нижнему регистру)
test_tokens = test_sentence.lower().split()

# Если число токенов меньше L, дополняем их пробелами; если больше, берем последние L токенов.
if len(test_tokens) < L:
    test_tokens = [' '] * (L - len(test_tokens)) + test_tokens
elif len(test_tokens) > L:
    test_tokens = test_tokens[-L:]

# Для каждого размера эмбеддингов оцениваем модель
for d in embeddings_d:
    # Загружаем модель, соответствующую текущей размерности эмбеддингов
    clear_keras_session()  # Очистка сессии Keras
    model = load_model(f"models/{dataset_name}_model_{d}_{'bpe' if bpe else 'word'}_emb.keras")
    
    # Загружаем эмбеддинги и словари
    embeddings_file = embeddings_path(dataset_name, d, bpe=bpe)
    embeddings, word2index, index2word = load_embeddings(embeddings_file)

    # Преобразуем токены в индексы
    test_indices = [word2index.get(word, 0) for word in test_tokens]
    test_indices_emb = embeddings[test_indices]
    
    print(f"Тестовые индексы: {test_indices}")
    
    
    # Предсказываем следующее слово по текущему контексту
    predicted = model.predict(np.array([test_indices_emb]))
    predicted_index = np.argmax(predicted)
    predicted_word = index2word[predicted_index]
    print(f"[{d}] Предсказанное следующее слово для '{' '.join(test_tokens).replace(' ' if bpe else '', '').replace('</w>', ' ')}': {predicted_word}")
    
    # Вычисляем перплексию для тестового предложения
    perplexity = evaluate_perplexity(model, test_sentence, L, word2index, embeddings)
    print(f"[{d}] Перплексия на тестовом предложении: {perplexity:.2f}")

    generated_words = []
    current_sequence = embeddings[test_indices]
    for _ in range(100):
        predicted = model.predict(np.array([current_sequence]), verbose=0)
        predicted_index = np.argmax(predicted)
        generated_words.append(index2word[predicted_index])
        current_sequence = np.vstack((current_sequence[1:], embeddings[predicted_index]))
    
    if bpe:
        s = ''.join(generated_words).split('</w>')
        print(f"Сгенерированные слова: ", *s)
    else:
        print(f"Сгенерированные слова: {' '.join(generated_words)}")


Тестовое предложение: 'ters</w> mind</w> you</w> pl enty</w> of</w> people</w> thought</w> he</w> was</w> going</w> about</w> things</w> the</w> right</w> way</w>'
Тестовые индексы: [69, 612, 0, 144, 171]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
[100] Предсказанное следующее слово для 'about things the right way ': to</w>
[100] Перплексия на тестовом предложении: 87.24
Сгенерированные слова:  to be sure that he had never seen it was a good idea of the wand and the wand and harry saw that he had never seen it was a good idea of the wand and the wand and harry saw that he had never seen it was a good idea of the wand and the wand and harry saw that he had never seen it was a good idea of the wand and the wand and harry saw that he had never seen it was a good idea of the wand and the wand and harry saw that he 
Тестовые индексы: [69, 612, 0, 144, 171]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
[500] Предсказанное следующее слово для 'about things the right way ': the</w>
[500] Перплексия на те